In [88]:
import os
import re
import json

# 替换替代模式为正常模式
def replace_alternative_patterns(content):
    # 替换section的替代模式
    content = re.sub(r'(?<!\w)section\s*"([^"]*?)"', r'section <open>\1<close>', content, flags=re.DOTALL)
    # 替换subsection的替代模式
    content = re.sub(r'(?<!\w)subsection\s*"([^"]*?)"', r'subsection <open>\1<close>', content, flags=re.DOTALL)
    # 替换subsubsection的替代模式
    content = re.sub(r'(?<!\w)subsubsection\s*"([^"]*?)"', r'subsubsection <open>\1<close>', content, flags=re.DOTALL)
    return content

# 按照section进行分割
def process_file(file_path):
    # 读取文件内容
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 替换替代模式为正常模式
    content = replace_alternative_patterns(content)
    
    # 用于匹配section、subsection和subsubsection的正则表达式模式
    section_pattern = re.compile(r'(?<!\w)section\s*\\?<open>(.*?)\\?<close>', re.DOTALL)
    subsection_pattern = re.compile(r'(?<!\w)subsection\s*\\?<open>(.*?)\\?<close>', re.DOTALL)
    subsubsection_pattern = re.compile(r'(?<!\w)subsubsection\s*\\?<open>(.*?)\\?<close>', re.DOTALL)

    # 获取文件名（不带扩展名）
    file_name = os.path.basename(file_path).replace('.thy', '')

    
    results = []
    current_section = ""
    current_subsection = ""
    current_subsubsection = ""

    # 分割section
    sections = section_pattern.split(content)
    for i in range(0, len(sections), 2):
        if i > 0:
            # 提取section名称
            match = section_pattern.search(content[len(''.join(sections[:i])):])
            if match:
                current_section = match.group(1).strip()

        subsections = subsection_pattern.split(sections[i])
        for j in range(0, len(subsections), 2):
            if j > 0:
                # 提取subsection名称
                match = subsection_pattern.search(sections[i][len(''.join(subsections[:j])):])
                if match:
                    current_subsection = match.group(1).strip()

            subsubsections = subsubsection_pattern.split(subsections[j])
            for k in range(0, len(subsubsections), 2):
                if k > 0:
                    # 提取subsubsection名称
                    match = subsubsection_pattern.search(subsections[j][len(''.join(subsubsections[:k])):])
                    if match:
                        current_subsubsection = match.group(1).strip()

                code = subsubsections[k].strip()
                if code:
                    results.append({
                        "title": file_path,
                        "section": current_section,
                        "subsection": current_subsection,
                        "subsubsection": current_subsubsection,
                        "code": '\n'+code
                    })

    return results

# 处理目录下的所有文件
def process_directory(directory):
    all_results = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.thy'):
                file_path = os.path.join(root, file)
                all_results.extend(process_file(file_path))
    
    return all_results

# 将处理结果保存为JSONL格式
def save_to_jsonl(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for entry in data:
            json.dump(entry, f)
            f.write('\n')

# 主函数
def main():
    directory = './spec'  # 需要处理的目录
    output_file = 'split_by_section.jsonl'  # 输出文件
    data = process_directory(directory)
    save_to_jsonl(data, output_file)

if __name__ == '__main__':
    main()

In [2]:
# DEPRECATED
import os
import re
import json

# 按照section进行分割
def process_file(file_path):
    # 读取文件内容
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 用于匹配section、subsection和subsubsection的正则表达式模式
    section_pattern = re.compile(r'(?<!\w)section\s*\\?<open>(.*?)\\?<close>', re.DOTALL)
    subsection_pattern = re.compile(r'(?<!\w)subsection\s*\\?<open>(.*?)\\?<close>', re.DOTALL)
    subsubsection_pattern = re.compile(r'(?<!\w)subsubsection\s*\\?<open>(.*?)\\?<close>', re.DOTALL)
    section_pattern_alt = re.compile(r'(?<!\w)section\s*"([^"]*?)"', re.DOTALL)
    subsection_pattern_alt = re.compile(r'(?<!\w)subsection\s*"([^"]*?)"', re.DOTALL)
    subsubsection_pattern_alt = re.compile(r'(?<!\w)subsubsection\s*"([^"]*?)"', re.DOTALL)

    # 首先按照section进行分割
    sections = section_pattern.split(content)
    sections_alt = section_pattern_alt.split(content)

    # 如果找到替代模式的匹配，则使用替代模式的分割结果
    if len(sections_alt) > 1:
        sections = sections_alt

    # 获取文件名（不带扩展名）
    file_name = os.path.basename(file_path).replace('.thy', '')
    
    results = []
    current_section = ""
    
    for i in range(len(sections)):
        if i % 2 == 0:
            # 这是代码段或第一个section之前的部分
            subsections = subsection_pattern.split(sections[i])
            subsections_alt = subsection_pattern_alt.split(sections[i])

            # 如果找到替代模式的匹配，则使用替代模式的分割结果
            if len(subsections_alt) > 1:
                subsections = subsections_alt

            current_subsection = ""
            for j in range(len(subsections)):
                if j % 2 == 0:
                    # 这是代码段或第一个subsection之前的部分
                    subsubsections = subsubsection_pattern.split(subsections[j])
                    subsubsections_alt = subsubsection_pattern_alt.split(subsections[j])

                    # 如果找到替代模式的匹配，则使用替代模式的分割结果
                    if len(subsubsections_alt) > 1:
                        subsubsections = subsubsections_alt

                    current_subsubsection = ""
                    for k in range(len(subsubsections)):
                        if k % 2 == 0:
                            # 这是代码段
                            code = subsubsections[k].strip()
                            if code:
                                results.append({
                                    "title": file_name,
                                    "section": current_section,
                                    "subsection": current_subsection,
                                    "subsubsection": current_subsubsection,
                                    "code": '\n'+code
                                })
                        else:
                            # 这是subsubsection名称
                            current_subsubsection = subsubsections[k].strip()
                else:
                    # 这是subsection名称
                    current_subsection = subsections[j].strip()
                    current_subsubsection = ""
        else:
            # 这是section名称
            current_section = sections[i].strip()
            current_subsection = ""
            current_subsubsection = ""
    
    return results

# 处理目录下的所有文件
def process_directory(directory):
    all_results = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.thy'):
                file_path = os.path.join(root, file)
                all_results.extend(process_file(file_path))
    
    return all_results

# 将处理结果保存为JSONL格式
def save_to_jsonl(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for entry in data:
            json.dump(entry, f)
            f.write('\n')

# 主函数
def main():
    directory = './spec'  # 需要处理的目录
    output_file = 'split_by_section.jsonl'  # 输出文件
    data = process_directory(directory)
    save_to_jsonl(data, output_file)

if __name__ == '__main__':
    main()


In [1]:
import json
import re

def process_json(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        chapter = ""
        # 用于判断前一条/后一条是否是一个文件
        pre_title = ""
        next_title = ""
        lines = infile.readlines()
        for i in range(len(lines)):
            line = lines[i]
            data = json.loads(line)
            if i>0:
                pre_data = json.loads(lines[i-1])
                pre_title = pre_data['title']
            else:
                pre_title = ""
            if i<len(lines)-1:
                next_data = json.loads(lines[i+1])
                next_title = next_data['title']
            else:
                next_title = ""
            

            # 清除subsubsection字段
            if 'subsubsection' in data:
                del data['subsubsection']

            
            if 'code' in data:
                # (*COMMENT*)格式的注释很多都包含Copyright等无效信息，因此需要先删掉
                data['code'] = re.sub(r'\(\*(.*?)(Copyright|FIXME|TODO)(.*?)\*\)', '', data['code'], flags=re.DOTALL)
                
                # 提取注释并分割code字段
                pattern = r'(?:\n\s*)(\(\*.*?\*\)|text\s*\\<open>.*?\\<close>|<comment>\s*\\<open>.*?\\<close>)'
                
                parts = re.split(pattern, data['code'].strip(), flags=re.DOTALL)                
                
                # 遍历每一部分
                for i in range(len(parts)):
                    part = parts[i].strip()
                    
                    # 如果当前部分是代码块
                    if i % 2 == 0:
                        code_part = part
                        if code_part == "":
                            continue

                        comment_part = parts[i-1].strip() if i > 0 else ""
                        # 提取注释
                        if comment_part:
                            comment = re.search(r'\(\*(.*?)\*\)|text\s*\\<open>(.*?)\\<close>|<comment>\s*\\<open>(.*?)\\<close>', comment_part, flags=re.DOTALL)
                            comment = comment.group(1) or comment.group(2) or comment.group(3)
                        else:
                            comment = ""

                        if code_part.startswith('theory') and code_part.endswith('begin'):
                            continue

                        
                        # 提取chapter
                        match = re.search(r'chapter(?:\s*"|\s*\\<open>)(.*?)(?:\s*"|\s*\\<close>)', code_part)
                            
                        cur_chapter = match.group(1) if match else ""
                        if cur_chapter:
                            chapter = cur_chapter

                            # 特殊情况：如果当前文件仅一个代码块，保留
                            if len(parts)==1 and data['title']!=next_title:
                                print(data['title'])
                                new_data = data.copy()
                                new_data['code'] = code_part
                                new_data['comment'] = comment
                                new_data['chapter'] = chapter
                                assert new_data['code']!=""
                                outfile.write(json.dumps(new_data, ensure_ascii=False) + '\n')

                            if comment:
                                chapter += f"\n(*{comment}*)"
                                
                            continue

                        # 如果当前代码块和上个代码块不属于同一个文件，清空chapter
                        elif data['title']!=pre_title:
                            chapter = ""

                        new_data = data.copy()
                        new_data['code'] = code_part
                        new_data['comment'] = comment
                        new_data['chapter'] = chapter
                        assert  new_data['code']!=""
                        outfile.write(json.dumps(new_data, ensure_ascii=False) + '\n')
                        

                        

                        # # 根据关键词分割code字段
                        # sub_parts = re.split(r'(?<=\n)(definition|fun|primrec|termination|context|setup|locale|local_setup|local|record|axiomatization|type_synonym|declare|unbundle|crunch_ignore|crunch|requalify_facts|global_interpretation|inductive_set|crunches|lemmas|lemma|method|inductive|match_abbreviation|abbreviation)(?=\s)', code_part)
                        # sub_parts = [sub_part.strip() for sub_part in sub_parts if sub_part.strip()]

                        # # 生成多个JSON对象
                        # if sub_parts and sub_parts[0] in ['definition','fun','primrec','termination','context','context','setup','locale','local_setup','local','record','axiomatization','type_synonym','declare','unbundle','crunch_ignore','crunch','requalify_facts','global_interpretation','inductive_set','crunches','lemmas','lemma','method','inductive','match_abbreviation','abbreviation','record']:
                        #     for j in range(0, len(sub_parts), 2):
                        #         new_data = data.copy()
                        #         new_data['code'] = sub_parts[j] + ' ' + (sub_parts[j+1] if j+1 < len(sub_parts) else '')
                        #         new_data['comment'] = comment if j == 0 else ""
                        #         if new_data['code']!="":
                        #             outfile.write(json.dumps(new_data, ensure_ascii=False) + '\n')
                        # else:
                        #     for j in range(-1, len(sub_parts), 2):
                        #         new_data = data.copy()
                        #         new_data['code'] = (sub_parts[j+1] if j+1 < len(sub_parts) else '') if j == -1 else sub_parts[j] + ' ' + (sub_parts[j+1] if j+1 < len(sub_parts) else '')
                        #         new_data['comment'] = comment if j == -1 else ""
                        #         if new_data['code']!="":
                        #             outfile.write(json.dumps(new_data, ensure_ascii=False) + '\n')

if __name__ == "__main__":
    process_json('./split_by_section.jsonl', './split_by_keywords2.jsonl')


./spec/design/skel/ObjectInstances_H.thy
./spec/design/skel/CNode_H.thy
./spec/design/skel/CSpace_H.thy
./spec/design/skel/ThreadDecls_H.thy
./spec/design/skel/Fault_H.thy
./spec/design/skel/RetypeDecls_H.thy
./spec/design/skel/NotificationDecls_H.thy
./spec/design/skel/Delete_H.thy
./spec/design/skel/Types_H.thy
./spec/design/skel/Structures_H.thy
./spec/design/skel/Notification_H.thy
./spec/design/skel/Endpoint_H.thy
./spec/design/skel/CSpaceDecls_H.thy
./spec/design/skel/Syscall_H.thy
./spec/design/skel/TCBDecls_H.thy
./spec/design/skel/Kernel_H.thy
./spec/design/skel/EndpointDecls_H.thy
./spec/design/skel/FaultHandlerDecls_H.thy
./spec/design/skel/KI_Decls_H.thy
./spec/design/skel/Untyped_H.thy
./spec/design/skel/ARM/ArchThreadDecls_H.thy
./spec/design/skel/ARM/ArchStateData_H.thy
./spec/design/skel/ARM/Arch_Structs_B.thy
./spec/design/skel/ARM/ArchVSpaceDecls_H.thy
./spec/design/skel/ARM/RegisterSet_H.thy
./spec/design/skel/ARM/ArchFaultHandler_H.thy
./spec/design/skel/ARM/ArchRet

In [4]:
# 【可跳过】单独提取有comment的数据
import json

def filter_non_empty_comments(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        for line in infile:
            try:
                data = json.loads(line)
                if 'comment' in data and data['comment'].strip() != "":
                    outfile.write(json.dumps(data, ensure_ascii=False) + '\n')
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                continue

if __name__ == "__main__":
    input_file = './split_by_keywords.jsonl'
    output_file = './record_with_comment.jsonl'
    filter_non_empty_comments(input_file, output_file)
    print(f"Filtered records with non-empty comments saved to {output_file}")


Filtered records with non-empty comments saved to ./record_with_comment.jsonl


In [ ]:
from http import HTTPStatus
from dashscope import Generation
import json
import time

def process_line(line, key, model):
    data = json.loads(line)
    title = data['title']
    chapter = data['chapter']
    section = data['section']
    subsection = data['subsection']
    comment = data['comment']
    code = data['code']

    # 构建用户输入
    user_input = f"""In operating system formalization, a specification is a detailed, formal description of the system's behavior, requirements, and constraints in code, serving as a blueprint for implementation and verification. Property should be abstract and concise in 1-3 sentence, you can learn from the comment if it is provided. 
Your task is to summarize specification property from code. You should answer question with one main property and optional subproperties(if the code is simple and one main property is enough to summarize, subproperties are unnecessary), without any extra explanation or statement like "Here are the properties:".  Please note that do not use the parts or fuction name in code to explain, just extract properties like that they are defined before writing spec code.
Here are examples:

Example1:
section: Thread Message Formats
comment: TCB capabilities confer authority to perform seven actions. A thread can request to yield its timeslice to another, to suspend or resume another, to reconfigure another thread, or to copy register sets into, out of or between
other threads.
code:
fun
  invoke_tcb :: "tcb_invocation \<Rightarrow> (data list,'z::state_ext) p_monad"
where
  "invoke_tcb (Suspend thread) = liftE (do suspend thread; return [] od)"
| "invoke_tcb (Resume thread) = liftE (do restart thread; return [] od)"

| "invoke_tcb (ThreadControl target slot faultep mcp priority croot vroot buffer)
   = doE
    liftE $ option_update_thread target (tcb_fault_handler_update o K) faultep;
    liftE $  case mcp of None \<Rightarrow> return()
     | Some (newmcp, _) \<Rightarrow> set_mcpriority target newmcp;
    (case croot of None \<Rightarrow> returnOk ()
     | Some (new_cap, src_slot) \<Rightarrow> doE
      cap_delete (target, tcb_cnode_index 0);
      liftE $ check_cap_at new_cap src_slot
            $ check_cap_at (ThreadCap target) slot
            $ cap_insert new_cap src_slot (target, tcb_cnode_index 0)
    odE);
    (case vroot of None \<Rightarrow> returnOk ()
     | Some (new_cap, src_slot) \<Rightarrow> doE
      cap_delete (target, tcb_cnode_index 1);
      liftE $ check_cap_at new_cap src_slot
            $ check_cap_at (ThreadCap target) slot
            $ cap_insert new_cap src_slot (target, tcb_cnode_index 1)
    odE);
    (case buffer of None \<Rightarrow> returnOk ()
     | Some (ptr, frame) \<Rightarrow> doE
      cap_delete (target, tcb_cnode_index 4);
      liftE $ thread_set (\<lambda>t. t \<lparr> tcb_ipc_buffer := ptr \<rparr>) target;
      liftE $ case frame of None \<Rightarrow> return ()
       | Some (new_cap, src_slot) \<Rightarrow>
            check_cap_at new_cap src_slot
          $ check_cap_at (ThreadCap target) slot
          $ cap_insert new_cap src_slot (target, tcb_cnode_index 4);
      cur \<leftarrow> liftE $ gets cur_thread;
      liftE $ when (target = cur) (do_extended_op reschedule_required)
    odE);
    liftE $ case priority
              of None \<Rightarrow> return()
               | Some (prio, _) \<Rightarrow> do_extended_op (set_priority target prio);
    returnOk []
  odE"

| "invoke_tcb (CopyRegisters dest src suspend_source resume_target transfer_frame transfer_integer transfer_arch) =
  (liftE $ do
    when suspend_source $ suspend src;
    when resume_target $ restart dest;
    when transfer_frame $ do
        mapM_x (\<lambda>r. do
                v \<leftarrow> as_user src $ getRegister r;
                as_user dest $ setRegister r v
        od) frame_registers;
        pc \<leftarrow> as_user dest getRestartPC;
        as_user dest $ setNextPC pc
    od;
    when transfer_integer $
        mapM_x (\<lambda>r. do
                v \<leftarrow> as_user src $ getRegister r;
                as_user dest $ setRegister r v
        od) gpRegisters;
    cur \<leftarrow> gets cur_thread;
    arch_post_modify_registers cur dest;
    when (dest = cur) (do_extended_op reschedule_required);
    return []
  od)"

| "invoke_tcb (ReadRegisters src suspend_source n arch) =
  (liftE $ do
    when suspend_source $ suspend src;
    self \<leftarrow> gets cur_thread;
    regs \<leftarrow> return (take (unat n) $ frame_registers @ gp_registers);
    as_user src $ mapM getRegister regs
  od)"

| "invoke_tcb (WriteRegisters dest resume_target values arch) =
  (liftE $ do
    self \<leftarrow> gets cur_thread;
    b \<leftarrow> arch_get_sanitise_register_info dest;
    as_user dest $ do
        zipWithM (\<lambda>r v. setRegister r (sanitise_register b r v))
            (frameRegisters @ gpRegisters) values;
        pc \<leftarrow> getRestartPC;
        setNextPC pc
    od;
    arch_post_modify_registers self dest;
    when resume_target $ restart dest;
    when (dest = self) (do_extended_op reschedule_required);
    return []
  od)"

| "invoke_tcb (NotificationControl tcb (Some ntfnptr)) =
  (liftE $ do
    bind_notification tcb ntfnptr;
    return []
  od)"

| "invoke_tcb (NotificationControl tcb None) =
  (liftE $ do
    unbind_notification tcb;
    return []
  od)"

| "invoke_tcb (SetTLSBase tcb tls_base) =
  (liftE $ do
    as_user tcb $ setRegister tlsBaseRegister tls_base;
    cur \<leftarrow> gets cur_thread;
    when (tcb = cur) (do_extended_op reschedule_required);
    return []
  od)"

definition
  set_domain :: "obj_ref \<Rightarrow> domain \<Rightarrow> unit det_ext_monad" where
  "set_domain tptr new_dom \<equiv> do
     cur \<leftarrow> gets cur_thread;
     tcb_sched_action tcb_sched_dequeue tptr;
     thread_set_domain tptr new_dom;
     ts \<leftarrow> get_thread_state tptr;
     when (runnable ts) (tcb_sched_action tcb_sched_enqueue tptr);
     when (tptr = cur) reschedule_required
   od"

definition invoke_domain:: "obj_ref \<Rightarrow> domain \<Rightarrow> (data list,'z::state_ext) p_monad"
where
  "invoke_domain thread domain \<equiv>
     liftE (do do_extended_op (set_domain thread domain); return [] od)"

Answer 1:
TCB Invocation Actions: Allows threads to perform actions such as suspending and resuming other threads, controlling various thread settings (fault endpoints, priorities, capability roots, IPC buffers), copying registers between threads, reading and writing thread registers, controlling notifications (binding and unbinding), and setting the TLS base. These actions ensure that threads can manage their state and capabilities effectively, maintaining the integrity and performance of the kernel.
Domain Management: Set the scheduling domain for a thread. It ensures that the thread is correctly dequeued, its domain is updated, and it is re-enqueued if runnable. If the thread being updated is the current thread, a reschedule is required. This ensures that threads are correctly managed within their scheduling domains, optimizing the kernel's scheduling and execution efficiency.

Example 2:
section: IPC Capability Transfers

comment: In addition to the data payload a message may also contain capabilities. When a thread requests additional capabilities be transferred the identities of those capabilities are retreived from the thread's IPC buffer.

code:
definition
  buffer_cptr_index :: nat
where
 "buffer_cptr_index \<equiv> (msg_max_length + 2)"

primrec
  get_extra_cptrs :: "obj_ref option \<Rightarrow> message_info \<Rightarrow> (cap_ref list,'z::state_ext) s_monad"
where
  "get_extra_cptrs (Some buf) mi =
    (liftM (map data_to_cptr) $ mapM (load_word_offs buf)
        [buffer_cptr_index ..< buffer_cptr_index + (unat (mi_extra_caps mi))])"
| "get_extra_cptrs None mi = return []"

definition
  get_extra_cptr :: "obj_ref \<Rightarrow> nat \<Rightarrow> (cap_ref,'z::state_ext) s_monad"
where
  "get_extra_cptr buffer n \<equiv> liftM data_to_cptr
      (load_word_offs buffer (n + buffer_cptr_index))"


Answer 2:
Get Extra Capability Pointers: Retrieve the additional capability pointers from the thread's IPC buffer. Load the word offsets corresponding to the extra capabilities and convert them to capability pointers.

Now, summarize property based on the following information. Don't use the information from the given example to summarize property.
"""

    if chapter:
        user_input += f"title:\n{chapter}\n\n"
    if section:
        user_input += f"section:\n{section}\n\n"
    if subsection:
        user_input += f"subsection:\n{subsection}\n\n"
    if comment:
        user_input += f"comment:\n{comment}\n\n"
    user_input += f"code:\n{code}"

    # 构建系统消息和用户消息
    messages = [
        {'role': 'system', 'content': 'You are a helpful assistant in operating system property generation.'},
        {'role': 'user', 'content': user_input}
    ]

    # 调用模型"llama3.1-405b-instruct"
    response = Generation.call(model=model, messages=messages, result_format='message',api_key=key)

    print(response)

    # 处理响应
    if response.status_code == HTTPStatus.OK:
        assistant_message = response.output.choices[0]['message']
        assert assistant_message['content']
        return {
            'spec': code,
            'property': assistant_message['content'],
            'title': title,
            'chapter':chapter,
            'section': section,
            'comment': comment
        }
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        return None

def main():
    # 读取jsonl文件
    with open('split_by_keywords.jsonl', 'r', encoding='utf-8') as file:
        json_lines = file.readlines()

    model = "llama3.1-405b-instruct"
    #model = "qwen-max"

    with open(f'output-{model}.jsonl', 'a', encoding='utf-8') as outfile:
        i = 0
        # 'sk-0599ccbef0814b3f8c4a8f5821c6d736'
        key = ['sk-a6c628b29e2f4b65b0ecd322f21e669f','sk-f4ebc969b87e41d7bf1dc91dd5026bba','sk-0fc3a02ca767436d91bea234f2f9d4e7','sk-de00e7083c4542458bfdcdc8158374a7','sk-68921001062244bd9d71139b015d1b9c']
        for line in json_lines[1214:]:      
            result = process_line(line, key[i%5], model)
            if result:
                json.dump(result, outfile, ensure_ascii=False)
                outfile.write('\n')
                i += 1
                time.sleep(3)
            else:
                break

if __name__ == '__main__':   
    main()

{"status_code": 200, "request_id": "941d2dfe-51f0-920f-a80b-33cb944a67e0", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "Set Global User Virtual Space: Switch to the global user address space by setting the virtual space root to the global user virtual space address with VMID 0."}}]}, "usage": {"input_tokens": 2052, "output_tokens": 33, "total_tokens": 2085}}
{"status_code": 200, "request_id": "0b9ed597-7924-93f3-ad30-f0fcc4b0a2b3", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "Address Space Management: Switch into the address space of a given thread or the global address space if none is correctly configured, ensuring that the thread's root page table is valid and properly configured.\n\nSubproperties:\n- Set VM Root: Set the virtual machine root for a given thread

In [15]:
from http import HTTPStatus
import dashscope
import os
import json
import time

def sample_sync_call(data):
    # 在这里设置问题
    prompt_text = f'''
你是一个数据清理助手。请清理文本中"Property:"等引导语，仅保留实际的属性。
文本1：Property: Define general-purpose registers within the machine architecture. 
回答1：Define general-purpose registers within the machine architecture. 
文本2：Property 1: Updates access rights of architecture capabilities while validating virtual memory rights.\nProperty 2: Defines architecture-specific object types including various page sizes and table structures.\nProperty 3: Represents interrupt request (IRQ) states for the x64 architecture with details for MSI and IOAPIC interrupts.
回答2：Updates access rights of architecture capabilities while validating virtual memory rights.Defines architecture-specific object types including various page sizes and table structures.Represents interrupt request (IRQ) states for the x64 architecture with details for MSI and IOAPIC interrupts.
你需要清理的文本：{data['property']}
请仅回答清理后的属性，不要回答任何其他内容，不包含开场白和结束语。
    '''
    response = dashscope.Generation.call(model='qwen2-1.5b-instruct',
                            prompt=prompt_text,
                            result_format='message')
    if response.status_code == HTTPStatus.OK:
        with open('cleaned_output.jsonl', 'a', encoding='utf-8') as outfile:
            assistant_message = response.output.choices[0]['message']
            # 将结果写入output.jsonl文件
            data['property'] = assistant_message['content']
            json.dump(data, outfile, ensure_ascii=False)
            outfile.write('\n')
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))

if __name__ == '__main__': 
    with open('output.jsonl', 'r', encoding='utf-8') as file:
        json_lines = file.readlines()
        count = 0
        for line in json_lines[703:]:
            if count%50 > 25:
                dashscope.api_key = 'sk-b2b3ccf68ce2480e94e0879f562115a8'
            else:
                dashscope.api_key = 'sk-4172100a27454844a6e3fbea414060a0'
            data = json.loads(line)
            sample_sync_call(data)
            count +=1
            time.sleep(0.5)



In [ ]:
user_input = f"""In operating system formalization, a specification is a detailed, formal description of the system's behavior, requirements, and constraints in code, serving as a blueprint for implementation and verification. Property should be abstract and concise in 1-3 sentence, you can learn from the comment if it is provided. 
Your task is to summarize specification property from code. You should answer question with one main property and optional subproperties(if the code is simple and one main property is enough to summarize, subproperties are unnecessary), without any extra explanation or statement like "Here are the properties:".  Please note that do not use the parts or function name in code to explain, just extract properties like that they are defined before writing spec code.
Here are examples:

Example1:
section: Thread Message Formats
comment: TCB capabilities confer authority to perform seven actions. A thread can request to yield its timeslice to another, to suspend or resume another, to reconfigure another thread, or to copy register sets into, out of or between
other threads.
code:
fun
  invoke_tcb :: "tcb_invocation \<Rightarrow> (data list,'z::state_ext) p_monad"
where
  "invoke_tcb (Suspend thread) = liftE (do suspend thread; return [] od)"
| "invoke_tcb (Resume thread) = liftE (do restart thread; return [] od)"
... 
...

Answer 1:
TCB Invocation Actions: Allows threads to perform actions such as suspending and resuming other threads, controlling...
Domain Management: Set the scheduling domain for a thread. It ensures...

Example 2:
...

Now, summarize property based on the following information. Don't use the information from the given example to summarize property.

In [1]:
import json

def clean_comments(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        previous_comment = None
        for line in infile:
            data = json.loads(line)
            if 'comment' in data:
                current_comment = data['comment']
                if current_comment == previous_comment:
                    data['comment'] = ''
                previous_comment = current_comment
            outfile.write(json.dumps(data) + '\n')

# 使用方法
input_file = 'cleaned_output.jsonl'
output_file = 'cleaned_output2.jsonl'
clean_comments(input_file, output_file)


In [16]:
from http import HTTPStatus
import dashscope
import os
import json
import time
def sample_sync_call(data):
    # 在这里设置问题
    prompt_text = f'''
你是一个操作系统的助手。请判断comment是否包含property中的大部分信息，如有关请回答yes，无关请回答no。
文本1：Property:Establishes the foundation for defining access rights within the system.\n
Comment:\nDefinition of access rights.\n 
回答1：yes
文本2：Property:Derive capabilities into a copyable form, handling various types with special cases, while ensuring no child capabilities exist for untyped caps, and returning null for non-copyable types.
Comment:Derive a cap into a form in which it can be copied. For internal reasons\nnot all capability types can be copied at all times and not all capability types\ncan be copied unchanged.
回答2：yes
文本3：Property:Define a constant identifier 'id0' with the value 0.
Comment:Proof to be done.\n
回答3：no
文本4：Property:Determine if a capability slot is the direct parent of another in the capability derivation tree.
Comment:to be used for abstraction unifying kernel objects other than TCB and CNode
回答4：no
你需要判断的内容：Property:{data['property']}
Comment:{data['comment']}\n
请仅回答yes或no，不要回答任何其他内容，不包含开场白和结束语。
    '''
    response = dashscope.Generation.call(model='qwen-turbo',
                            prompt=prompt_text,
                            result_format='message')
    if response.status_code == HTTPStatus.OK:
        with open('cleaned_output3.jsonl', 'a', encoding='utf-8') as outfile:
            assistant_message = response.output.choices[0]['message']
            print(assistant_message)
            # 将结果写入output.jsonl文件
            if assistant_message['content']=='no':
                data['comment'] = ''
            json.dump(data, outfile, ensure_ascii=False)
            outfile.write('\n')
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))

if __name__ == '__main__': 
    with open('cleaned_output2.jsonl', 'r', encoding='utf-8') as file:
        json_lines = file.readlines()
        count = 0
        for line in json_lines[:]:
            if count%50 > 25:
                dashscope.api_key = 'sk-b2b3ccf68ce2480e94e0879f562115a8'
            else:
                dashscope.api_key = 'sk-4172100a27454844a6e3fbea414060a0'
            data = json.loads(line)
            if data['comment']:
                sample_sync_call(data)
                count +=1
                time.sleep(0.5)
            else:
                with open('cleaned_output3.jsonl', 'a', encoding='utf-8') as outfile:
                    json.dump(data, outfile, ensure_ascii=False)
                    outfile.write('\n')

{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "no"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{"role": "assistant", "content": "yes"}
{